# Analysis of Intel Stock Prices

### Alan Elbert

##  Introduction 

Predicting a stocks price is a multibillion dollar industry, with numerous hedgefunds investing massive sums of money in order to determine what will happen to a stock's price. Thousands, if not millions of factors affect the price of a stock, ranging from the overall performance of the economy, public sentiment, commodity prices, geopolitical events, and much more.

The overall objective of this project will be to analyze the prices of Intel stock, specifically in comparison to commodity price indices and the performance of other stocks related to Intel, mainly its competitors and companies that Intel does business with (Suppliers and Customers of Intel). Although this will come no where near the scale of analysis done at larger firms with more resources, I hope that this will reveal at least a few interesting trends, that a budding trader might consider when deciding to buy, or sell, Intel stock.



### Required Libraries

You will need the following libraries and all their dependencies for this project

1. Pandas
2. Numpy
3. ScikitLearn
4. Statsmodels
5. Matplotlib
6. Datetime
7. Scipy
8. Requests
9. json


In [1]:

import pandas as pd
import numpy as np
import urllib
import requests
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sma
from scipy import stats
import statsmodels.api as sm
import math
from datetime import date
import os.path
import json
from datetime import date

# Warnings about the future kept clogging up the output
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

/home/alan/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/home/alan/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


# Data Acquisition and Processing

This is the data acquisition and wrangling phase. To make it easier to follow along, we are going to combine both parts into one for every source of data

For our analysis today, we will be getting two types of data, stock and commodity price data. Getting each will require the use of different libraries and different techniques to wrangle for use later.

## 1a Stock Data

### Data Acquisition

We want to compare Intel's stock prices to various other companies in the industry related to Intel. We are mainly going to focus on four different kinds of companies, Intel (INTC) itself, Intel's suppliers, Intel's customers, and Intel's competitors. We will only use data from after 2001, because in 2000 there was a massive stock crash involving numerous tech comapanies (the dot com bubble), which for us represents a massive outlier.

If you read this massive financial statement, linked below, several insights can be found about what comapnies we should focus on for our analysis:

https://www.intc.com/filings-reports/annual-reports/content/0000050863-22-000007/0000050863-22-000007.pdf

Firstly, Intel's three largest customers, Dell (DELL), Lenovo (LNVGF), and HP Inc (HPE) , accounted for 43% of Intel's revenue. However, this report unfortunately does not include any mention of smaller customers, but for the purposes of this tutorial, especially when they take up such a large portion of Intel's revenue, they will be enough.

As for suppliers, a list of Intel's top suppliers can be found here:

https://www.intc.com/news-events/press-releases/detail/1455/intel-2020-supplier-continuous-quality-improvement-awards

For the purposes of this analysis, we will be focusing on Intel's top suppliers, namely those who received the SCQI award. As such, they are 

* Applied Materials, Inc (AMAT)
* Lam Research Corporation (LRCX)
* Securitas Security Services USA, Inc (SECU-B.ST)
* Siltronic AG (WAF.DE)
* Taiwan Semiconductor Manufacturing Company, Limited (TSM)
* Tokyo Electron Limited (TOELY)

Additionally, looking at the page, you will notice that these suppliers represent diverse and varied industries, randing from silicon wafers to solder paste, and will hopefully paint the best possible picture of the performance of Intel's suppliers.

Finally, as for the main competitors, a list of such companies can be found here, from Investopedia, which are namely:

* AMD (AMD)
* IBM (IBM)
* NVIDIA (NVDA)
* Samsung (SMSN.IL)

Hopefully, by analyzing the stocks of all these companies related to Intel, we will find some interesting relationships that will allow us to somewhat predict the its stock price.

In order to do this, we will use Yahoo's Finance API along with requests and json libraries in order to request the information we need directly from Yahoo's database through the Spark API endpoint. More information on how to use the Finance API with the requests library can be found in the links below:

https://www.yahoofinanceapi.com/tutorial
https://www.yahoofinanceapi.com/


Another way to get Yahoo's finance data is to go directly to Yahoo's site and download the data as a .csv file. This is somewhat tedious however, and does not scale well if you want to analyze additional companies. In this tutorial, we will do both. For Intel's stock data, we will download it directly in order to demonstrate the latter method using the link below, but for every other companies' stock data we will go the requests route.

https://finance.yahoo.com/quote/intc/history/

(Use the download link under the apply button)

In [2]:

# This method returns a JSON containing data for the company(s) requested
def get_historical_data(symbols):

    # We want to use the spark endpoint
    url = "https://yfapi.net/v8/finance/spark"

    # Automatically generates the request we need
    querystring = {"symbols":symbols, "range":"21y", "interval":"1mo"}

    # This is the key associated with my personal free account. If this does not work for some reason, you can make your own account using the links above, and use your own key
    headers = {
        'x-api-key': "PKGDrT01fX36RI0yCCEbfaPqLfqvToj71o3reBkp"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    return response.json()



customers = ['DELL','LNVGF','HPE']

# I omitted LCRX as there was a strange error downloading the json file
suppliers = ['AMAT', 'SECU-B.ST','WAF.DE','TSM','TOELY']
competitors = ['AMD','IBM','NVDA','SMSN.IL']

everyone = customers + suppliers + competitors


# The free API has limited requests per day. So, if the JSON already exists, there is no point in wasting precious API calls. So, as soon as we get the JSON we save it locally
# Another issue I found was that although the API allows you to request data for up to 10 companies at a time, there is somewhat of a cap on JSON size, which caused missing data
# To alleviate this, we can just request the data for companies individually
for n in everyone:

    if not os.path.exists(n + ".json"):
        print("Downloading: " + n)
        data = get_historical_data(n)
        with open(n + ".json", 'w') as f:
            json.dump(data, f)


f = open('AMD.json')

toprint = json.load(f)

print(json.dumps(toprint, indent=4, sort_keys=True))



{
    "AMD": {
        "chartPreviousClose": 27.68,
        "close": [
            28.9,
            18.26,
            13.55,
            8.15,
            9.84,
            13.56,
            15.86,
            16.05,
            13.5,
            14.71,
            11.18,
            11.43,
            9.72,
            8.03,
            8.85,
            5.34,
            6.14,
            9.0,
            6.46,
            5.24,
            5.49,
            6.18,
            7.44,
            7.28,
            6.41,
            7.3,
            11.29,
            11.11,
            15.2,
            17.97,
            14.9,
            14.86,
            15.0,
            16.23,
            14.22,
            15.55,
            15.9,
            12.49,
            11.43,
            13.0,
            16.82,
            21.28,
            22.02,
            15.8,
            17.45,
            16.12,
            14.23,
            16.4,
            17.34,
            20.08,
      

Now that we have gotten all the data downloaded from Yahoo's API in .json format, an example of which is shown above, the next step is to load in Intel's data. This was manually downloaded from the website (link above) and transfered to the tutorial folder. Once it is locally available, loading it is as simple as using pandas' .load_csv() function.

In [3]:
intcdf = pd.read_csv('INTC.csv')
intcdf.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2002-05-01,28.570000,31.360001,25.559999,27.620001,16.863897,976795600
1,2002-06-01,27.430000,28.200001,17.450001,18.270000,11.163094,1334010600
2,2002-07-01,18.350000,19.879999,16.260000,18.790001,11.480818,1507029200
3,2002-08-01,18.719999,19.670000,15.820000,16.670000,10.185485,1182004600
4,2002-09-01,16.469999,17.040001,13.670000,13.890000,8.497054,1253168200


### Data Processing

There is a glaring issue that comes up when you examine stock prices across years. A drop from 100$ to 98$ is not as significant for investors as say a drop from 10$ to 8$. If you invested 100$ in the former, you would only lose 2$, but investing in the latter would lose you 20$. All of these stocks go through large swings in their prices over the past twenty years, so just computing the change in dollars between the open and close for every month would simply be misleading and innaccurate. In order to enable us to compare changes in prices across varying price levels, we are going to compute the percent change instead.

For the Intel dataframe, this process is quite simple, we can just subtract the open price column from the close price column, divide it by the open price column, and then multiply by 100 to get our percent change.

In [4]:
intcdf["percent_change"] = ((intcdf["Close"] - intcdf["Open"]) / intcdf["Open"]) * 100.0
intcdf.head()

,Date,Open,High,Low,Close,Adj Close,Volume,percent_change
0,2002-05-01,28.570000,31.360001,25.559999,27.620001,16.863897,976795600,-3.325163
1,2002-06-01,27.430000,28.200001,17.450001,18.270000,11.163094,1334010600,-33.394094
2,2002-07-01,18.350000,19.879999,16.260000,18.790001,11.480818,1507029200,2.397826
3,2002-08-01,18.719999,19.670000,15.820000,16.670000,10.185485,1182004600,-10.950850
4,2002-09-01,16.469999,17.040001,13.670000,13.890000,8.497054,1253168200,-15.664840


Additionally, now that we have computed the percent change, the other columns are quite useless for our analysis, and keeping them will only cause bloat and confusion, so we will drop them using the following line of code.

In [5]:
intcdf.drop(intcdf.columns.difference(['Date', 'percent_change']), 1, inplace=True)
intcdf.head()

,Date,percent_change
0,2002-05-01,-3.325163
1,2002-06-01,-33.394094
2,2002-07-01,2.397826
3,2002-08-01,-10.950850
4,2002-09-01,-15.664840


We have now processed the Intel data into a format that we can use for further analysis. Now the tricky part begins, dealing with the JSON data. Unlike a .csv which is easily readable by pandas, a json is not. Although pandas does have a read_json function, looking at the output below for AMD.json, for example, it simply is not formatted correctly.

In [6]:
amd_t = pd.read_json('AMD.json', orient='index')
amd_t.head()

,symbol,timestamp,dataGranularity,close,end,start,previousClose,chartPreviousClose
AMD,AMD,"[991368000, 993960000, 996638400, 999316800, 1...",300,"[28.9, 18.26, 13.55, 8.15, 9.84, 13.56, 15.86,...",NaN,NaN,NaN,27.68


However, looking at the output above, the data we need is loaded, we just have to extract it and reformat it properly. We will do this by first creating a new empty dataframe, and then iterating through the arrays storing the timestamp and close data to add them to the new dataframe one row at a time.

In [7]:

amd = pd.DataFrame()

i = 0

# Getting the arrays we need
tstamp = amd_t['timestamp'][0]
close = amd_t['close'][0]

# Adding rows to the df one at a time
for n in tstamp:
    amd.at[i, 'Date'] = n
    amd.at[i, 'Close'] = close[i]
    i += 1


amd.head()

,Date,Close
0,9.913680e+08,28.90
1,9.939600e+08,18.26
2,9.966384e+08,13.55
3,9.993168e+08,8.15
4,1.001909e+09,9.84


Now we have something that is formatted correctly, which will allow us to do the next phase of data wrangling. Another way to do this is to use the JSON parser instead of pandas to extract the relevant values, as shown below.

In [8]:
# Opening the JSON as a file
f = open('AMD.json')

# This reads the file, parses it, and returns a dictionary with all the necessary data
amd_jdata = json.load(f)


print("First ten timestamps:")
print(amd_jdata['AMD']['timestamp'][0:10])

print("First ten close prices:")
print(amd_jdata['AMD']['close'][0:10])

First ten timestamps:
[991368000, 993960000, 996638400, 999316800, 1001908800, 1004590800, 1007182800, 1009861200, 1012539600, 1014958800]
First ten close prices:
[28.9, 18.26, 13.55, 8.15, 9.84, 13.56, 15.86, 16.05, 13.5, 14.71]


From here, the data can be simply added to a dataframe with the loop above. Now, we have to further clean up the data. Firstly, although UNIX time is very readable to machines, we are not machines but inferior creatures of flesh and bone, and in order to make the time stamps more readable to humans we must convert them int YYYY-MM-DD format, or something similar. Fortunately, the date time library happens to come with a handy .fromtimestamp() method, which we can use on the data as show below:

In [9]:
amd['Date'] = [ date.fromtimestamp(int(ts)) for ts in amd['Date'] ]

amd.head()

,Date,Close
0,2001-06-01,28.90
1,2001-07-01,18.26
2,2001-08-01,13.55
3,2001-09-01,8.15
4,2001-10-01,9.84


Now, for the final part of , creating a percent change column. Unlike the dataframe from the Intel CSV, we have no convinient open column for us to use, only the close column. What we can use instead is the previous months data. This will result in us having no percent change for the first month, therefore resulting in the loss of one row of data unfortunately. We can accomplish this by iterating through all thr rows, and computing the same formula we used to calculate the percent change for Intel stock. So,

In [10]:

for i, row in amd.iterrows():
    
    # We can not compute percent change for the first row
    if i == 0:
        amd.at[i, 'percent_change'] = np.nan
    else:

        amd.at[i, 'percent_change'] = 100*(row['Close'] -  amd.at[(i- 1), 'Close'])/(amd.at[(i - 1, 'Close')])


amd.dropna(inplace=True)
amd.drop(['Close'], 1, inplace=True)
amd.head()

,Date,percent_change
1,2001-07-01,-36.816609
2,2001-08-01,-25.794085
3,2001-09-01,-39.852399
4,2001-10-01,20.736196
5,2001-11-01,37.804878


Finally, we have a Dataframe that resembles the one we made for Intel earlier. Now we will put all the code we used for the AMD json into a function in order to make a dataframe for every JSON we have downloaded.

In [11]:

# This function is just a compilation of code we wrote earlier for loading the contents of a json into a DF, and preprocessing
def df_fromjson(name):
    
    toret_t = pd.read_json(name + '.json', orient='index')

    toret = pd.DataFrame()

    i = 0

    # Getting the arrays we need
    tstamp = toret_t['timestamp'][0]
    close = toret_t['close'][0]

    # Adding rows to the df one at a time
    for n in tstamp:
        toret.at[i, 'Date'] = n
        toret.at[i, 'Close'] = close[i]
        i += 1

    # Making the date human readable
    toret['Date'] = [ date.fromtimestamp(int(ts)) for ts in toret['Date'] ]


    # Computing percent change
    for i, row in toret.iterrows():
    
        # We can not compute percent change for the first row
        if i == 0:
            toret.at[i, 'percent_change'] = np.nan
        else:

            toret.at[i, 'percent_change'] = 100*(row['Close'] -  toret.at[(i- 1), 'Close'])/(toret.at[(i - 1, 'Close')])


    # Removing unneeded columns and the first row
    toret.dropna(inplace=True)
    toret.drop(['Close'], 1, inplace=True)

    return toret


# Testing the function to make sure it does everything correctly
test = df_fromjson('AMD')
test.head()

,Date,percent_change
1,2001-07-01,-36.816609
2,2001-08-01,-25.794085
3,2001-09-01,-39.852399
4,2001-10-01,20.736196
5,2001-11-01,37.804878


Having confirmed that our conversion function works, we can now go ahead and apply it to all the JSONs. In order to store the many dataframes, we will use a hash table with the name of the company as a string, and the value being the dataframe.

In [12]:
# The hash table holding all the data frames
dfs = {}

# Creating the DFs for every JSON
for n in everyone:
    temp = df_fromjson(n)
    dfs[n] = temp


# Testing if it made of DF correctly
for n in everyone:
    print(n + ":")
    print(dfs[n].head())

DELL:
         Date  percent_change
1  2016-10-01        2.698472
2  2016-11-01        9.102127
3  2016-12-01        2.634555
4  2017-01-01       14.591301
5  2017-02-01        0.791945
LNVGF:
         Date  percent_change
1  2009-04-01       27.272727
2  2009-05-01       39.285714
3  2009-06-01        0.000000
4  2009-07-01       17.948718
5  2009-08-01       -6.521739
HPE:
         Date  percent_change
1  2015-12-01        2.280885
2  2016-01-01       -9.474757
3  2016-02-01       -3.551332
4  2016-03-01       33.605601
5  2016-04-01       -6.035905
AMAT:
         Date  percent_change
1  2001-07-01       -6.598778
2  2001-08-01       -6.040122
3  2001-09-01      -33.998608
4  2001-10-01       19.936709
5  2001-11-01       16.505424
SECU-B.ST:
         Date  percent_change
1  2001-06-30        2.077922
2  2001-07-31      -17.048346
3  2001-08-31        3.374233
4  2001-09-30        5.341246
5  2001-10-31        7.323944
WAF.DE:
         Date  percent_change
1  2015-07-31      -24.0121

Now, we are done the data acquisition and wrangling process for the stock data, and ready to do analysis on it!

## 1b  Commodity Price data 

### Acquisition

For this part of the analysis, we will download the commodity price data for every month from 2002 to 2021, to perform analysis on it later. Given the scale and complexity of global supply chains, we will use all the data we have. It might be difficult to see how the price of fertilizer might possibly affect the stock price of Intel, and we might find out later there is no relationship, but if we do not test it out we might be missing some really interesting relationships.

In comparison to acquiring the stock data from Yahoo finance, acquiring commodity price data for several different commodities is as simple as downloading a file. Pandas' .read_excel() can automatically download and read the file containing the data we need from a provided link.


In [13]:

# While working on the final tutorial this link went dark and the data set dissappeared into oblivion
# Fortunately I had a local copy stored on my machine, but here is how you would have done it if the link was still active
# Contact me at aelbert@terpmail.umd.edu if you want the dataset used
# tcomm = pd.read_excel('https://www.imf.org/-/media/Files/Research/CommodityPrices/Monthly/external-data-indices-onlyapril.ashx')

# We are using a local copy since the link went dead
tcomm = pd.read_excel('external-data-indices-onlyapril.xls')
tcomm.head()

,Commodity,PALLFNF,PEXGALL,PNFUEL,PFANDB,PFOOD,PBEVE,PINDU,PAGRI,PRAWM,PALLMETA,PMETA,PPMETA,PEXGMETA,PFERT,PNRG,POILAPSP,PNGAS,PCOAL
0,Commodity.Description,"All Commodity Price Index, 2016 = 100, include...","Commodities for Index: All, excluding Gold, 20...","Non-Fuel Price Index, 2016 = 100, includes Pre...","Food and Beverage Price Index, 2016 = 100, inc...","Food Price Index, 2016 = 100, includes Cereal,...","Beverage Price Index, 2016 = 100, includes Cof...","Industrial Inputs Price Index, 2016 = 100, inc...","Agriculture Price Index, 2016 = 100, includes ...","Agricultural Raw Materials Index, 2016 = 100, ...","All Metals Index, 2016 = 100: includes Metal P...","Base Metals Price Index, 2016 = 100, includes ...","Precious Metals Price Index, 2016 = 100, inclu...","All Metals EX GOLD Index, 2016 = 100: includes...","Fertilizer Index, 2016 = 100, includes DAP, Po...","Fuel (Energy) Index, 2016 = 100, includes Crud...","Crude Oil (petroleum), Price index, 2016 = 100...","Natural Gas Price Index, 2016 = 100, includes ...","Coal Price Index, 2016 = 100, includes Austral..."
1,Data Type,Index,Index,Index,Index,Index,Index,Index,Index,Index,Index,Index,Index,Index,Index,Index,Index,Index,Index
2,Frequency,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly,Monthly
3,1990M1,NaN,NaN,NaN,NaN,NaN,49.20388,NaN,NaN,72.855085,NaN,NaN,33.097764,NaN,NaN,NaN,NaN,NaN,51.172528
4,1990M2,NaN,NaN,NaN,NaN,NaN,49.947777,NaN,NaN,73.449658,NaN,NaN,33.701146,NaN,NaN,NaN,NaN,NaN,51.172528


### Data Processing


Keeping in line with the same rationale for computing percent change across the years, we want to do the same for commodity prices, as well, in addition to cleaning up the data so it is in line with the stock data too. However, in order to know what must be cleaned up in the data, we must first observe the data.

Looking at the above data, you will notice several glaring issues. Firstly, three rows do not carry any useful data at all. They must be dropped from the dataframe. Secondly, notice the Commodity column, specifically the format the data is in. In order to match up, and do analysis on the commodity data in relation to the stock data, this column must be replaced with datetime objects instead of simple string data. Furthermore, notice the NaN columns. There are several ways of dealing with them, such as dropping columns with NaN values, or simply ignoring NaN's for the final analysis. 

In [14]:



# We are removing the first 3 rows which carry no useful information. We can do this using .drop()

tcomm = tcomm.drop([0,1,2])


# This is just a bit of legacy code I kept in to only select a few columns, used for debugging purposes. I left it in because I used the array later and did 
# not want to break anything by removing it. 

tpchange = ['PALLFNF',	'PEXGALL',	'PNFUEL',	'PFANDB',	'PFOOD', 'PBEVE',	
    'PINDU', 'PAGRI', 'PRAWM',	'PALLMETA',	'PMETA', 'PPMETA', 'PEXGMETA',	'PFERT', 'PNRG', 'POILAPSP', 'PNGAS', 'PCOAL']

# We can also drop columns, and do it in place as well
tcomm.drop(tcomm.columns.difference(['Commodity'] + tpchange), 1, inplace=True)


# This loop converts the YEARM(MONTH) string into a form pythons built in date time parser can parse, which allows us to compare date times directly
# .iterrows() iterates per row, giving us its index and the contents of each row for us to look at
for i, row in tcomm.iterrows():
    datestr = (row['Commodity'])

    yearstr = datestr[0:4]
    mstr = datestr[-2:]
    mstr = mstr.replace("M", "0")

    isostr = yearstr + "-" + mstr + "-01"
    
    # In order to update the date, we have to use .at[index, column] to update the DF directly
    tcomm.at[i, 'Commodity'] = date.fromisoformat(isostr)

# Drops all rows we do not need
tcomm = tcomm[tcomm['Commodity'] >= date.fromisoformat('2002-04-01')]

# Ensures all numbers are floats

for i, row in tcomm.iterrows():
    for n in tpchange:
        tcomm.at[i, n] = float(row[n])

# We will calculate percent change using a deep copy which has one less row

commdata = tcomm.copy(deep=True)
commdata = commdata.drop([150])


# Resets the index so that we can compare across dataframes, also removes the previously created index
tcomm = tcomm.reset_index()
tcomm.drop(['index'], 1, inplace=True)

commdata = commdata.reset_index()
commdata.drop(['index'], 1, inplace=True)


# Computes percent changew
for i, row in commdata.iterrows():
    for n in tpchange:
        commdata.at[i, n + "_PCHANGE"] = 100 * ( (commdata.at[i,n] - tcomm.at[i,n]) / tcomm.at[i,n])

# Now that percent change has been computed, we no longer need the original prices
commdata.drop(tpchange, 1, inplace = True)

commdata.head()

,Commodity,PALLFNF_PCHANGE,PEXGALL_PCHANGE,PNFUEL_PCHANGE,PFANDB_PCHANGE,PFOOD_PCHANGE,PBEVE_PCHANGE,PINDU_PCHANGE,PAGRI_PCHANGE,PRAWM_PCHANGE,PALLMETA_PCHANGE,PMETA_PCHANGE,PPMETA_PCHANGE,PEXGMETA_PCHANGE,PFERT_PCHANGE,PNRG_PCHANGE,POILAPSP_PCHANGE,PNGAS_PCHANGE,PCOAL_PCHANGE
0,2002-05-01,NaN,NaN,NaN,-1.365817,-1.164610,-4.766974,-0.545688,-1.104417,0.609014,0.446983,-1.233882,3.025502,-1.163527,NaN,1.244313,-0.611939,3.208914,-2.386656
1,2002-06-01,NaN,NaN,NaN,-0.923893,-0.968423,-0.142708,3.302399,-0.108400,5.132082,2.088413,2.191574,1.936703,2.070231,NaN,-4.642933,-6.353787,-3.861909,-6.126784
2,2002-07-01,NaN,NaN,NaN,3.340158,3.273896,4.492992,0.970699,3.489521,4.394058,-1.731508,-1.167473,-2.563065,-1.376520,NaN,3.633266,2.768587,2.483602,-6.666984
3,2002-08-01,NaN,NaN,NaN,-0.182643,-0.278322,1.462563,-1.692926,0.143315,2.097384,-2.898913,-4.193506,-0.962961,-3.838795,NaN,1.629957,4.681271,-5.000659,-5.057825
4,2002-09-01,NaN,NaN,NaN,1.502661,0.927666,11.220133,1.775284,1.920696,4.370782,1.014690,-0.049478,2.554153,0.096096,NaN,6.664290,5.577429,8.432123,5.180339


Now, we have gotten our commodity data formatted into 